In [1]:
!pip3 install -q --upgrade pip gdown onnxruntime efficientnet-pytorch

In [2]:
import numpy as np
import pandas as pd
from PIL import Image, ImageOps

In [3]:
import torch
import onnxruntime
from torch import nn
from efficientnet_pytorch import EfficientNet

In [4]:
class Net(nn.Module):
    
    def __init__(self, n_meta_features):
        
        super(Net, self).__init__()
        
        self.arch = EfficientNet.from_pretrained('efficientnet-b1')
        self.arch._fc = nn.Linear(1280, 500, bias=True)
        self.arch.set_swish(memory_efficient=False)
        
        self.meta = nn.Sequential(
            nn.Linear(n_meta_features, 500),
            nn.BatchNorm1d(500),
            nn.ReLU(),
            nn.Dropout(p=0.2),
            nn.Linear(500, 250),
            nn.BatchNorm1d(250),
            nn.ReLU(),
            nn.Dropout(p=0.2))
        
        self.ouput = nn.Linear(500 + 250, 1)
        
    def forward(self, x, meta):
        cnn_features = self.arch(x)
        meta_features = self.meta(meta)
        features = torch.cat((cnn_features, meta_features), dim=1)
        output = self.ouput(features)
        return output

In [13]:
path = 'model.pth' 
dev = torch.device('cpu')
model = Net(n_meta_features=12)
model.load_state_dict(torch.load(path, map_location=dev))

Loaded pretrained weights for efficientnet-b1


<All keys matched successfully>

In [6]:
meta = torch.rand(1, 12)
x = torch.rand(1, 3, 256, 256)
model = Net(n_meta_features=12).eval()
nn.Sigmoid()(model(x, meta))

Loaded pretrained weights for efficientnet-b1


tensor([[0.5424]], grad_fn=<SigmoidBackward>)

In [7]:
torch.onnx.export(model, (torch.rand(1,3,256,256), torch.rand(1,12)), 'model.onnx')

In [11]:
def sigmoid(x): return 1 / (1 + np.exp(-x))

In [12]:
onnx_model = onnxruntime.InferenceSession('model.onnx')
meta = np.random.randn(1, 12).astype(np.float32)
x = np.random.randn(1, 3, 256, 256).astype(np.float32)
inp0_name = onnx_model.get_inputs()[0].name
inp1_name = onnx_model.get_inputs()[1].name
inps = {inp0_name: x, inp1_name: meta}
outs = onnx_model.run(None, inps)
sigmoid(outs[0])[0][0]

0.56362355

In [13]:
model

Net(
  (arch): EfficientNet(
    (_conv_stem): Conv2dStaticSamePadding(
      3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False
      (static_padding): ZeroPad2d(padding=(0, 1, 0, 1), value=0.0)
    )
    (_bn0): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
    (_blocks): ModuleList(
      (0): MBConvBlock(
        (_depthwise_conv): Conv2dStaticSamePadding(
          32, 32, kernel_size=(3, 3), stride=[1, 1], groups=32, bias=False
          (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
        )
        (_bn1): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
        (_se_reduce): Conv2dStaticSamePadding(
          32, 8, kernel_size=(1, 1), stride=(1, 1)
          (static_padding): Identity()
        )
        (_se_expand): Conv2dStaticSamePadding(
          8, 32, kernel_size=(1, 1), stride=(1, 1)
          (static_padding): Identity()
        )
        (_project_con